In [ ]:
import pandas as pd
import sqlalchemy
import pyodbc
import os
from pw import *

In [ ]:
from sqlalchemy.engine import URL
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": pna_connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

from sqlalchemy.types import NVARCHAR
cursor = engine.connect()

In [ ]:
## Set brand name.
brand = 'CT'
## Set catalog run name.
catalog = 'WN24'
## Set the start/in home date.
startdate = '01/08/2024'

# Auto-setting brand name.
if brand == "CT":
    name = "Connecting Threads (CT)"
elif brand == "KP":
    name = "Knit Picks (KP)"
elif brand == "ST":
    name = "Superior Threads (ST)"
elif brand == "WC":
    name = "We Crochet (WC)"

In [ ]:
csv = pd.read_csv(r'./Processing/'+brand+'_'+catalog+'.csv', dtype = object)

In [ ]:
# Just pull in all item ids and their internal ids. 
# It's a small list, not very inefficient. 
items = (
"""
SELECT 
    itemid,
    id as internalid
FROM NetSuite.ns.Item
"""
)

netsuite = pd.read_sql(items, engine)

netsuite['internalid'] = netsuite['internalid'].astype(str)

In [ ]:
df = csv.merge(netsuite, left_on='Item', right_on='itemid', how='inner').reset_index(drop=True)

In [ ]:
# Create a list of counts with 6-digit places depending on file size.
total_rows = len(df)
counts = [str(i).zfill(4) for i in range(1, total_rows + 1)]

# Create External ID column.
df['externalid'] = brand + '-' + catalog + '-PG-' +pd.Series(counts)

# Add in home date.
df['inhome'] = startdate

# Add catalog code.
df['catalog'] = catalog

# Add brand name.
df['brand'] = name

# Drop extraneous columns not needed for import.
df = df.drop(['Item', 'itemid', 'Brand', 'Catalog'], axis=1)

In [ ]:
# Create US mail files, put in applicable folders.
folder_path = os.path.join('Exports', brand, catalog, 'Pagination')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_path = os.path.join(folder_path, f'{brand}_{catalog}.csv')

df.to_csv(file_path, index=False)

In [ ]:
# Find missing or typo item ids.
left_join_df = csv.merge(netsuite, left_on='Item', right_on='itemid', how='left')
# Filter to get only rows where 'itemid' is null
missing_df = left_join_df[left_join_df['itemid'].isnull()]

missing_df.to_csv('missing_data.csv', index=False)